# Data Understanding

a).Specifying the Question

Performing sentiment Analysis on people's views towards Remote learning in Kenya using Machine Learning Models.

b).Defining the Metric for Success

Our project will be considered successful if we can effectively build a model that can give an accuracy of more than 80%.

c).Understanding the Context

## Importing our libraries

In [ ]:
#For data exploration
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# modelling
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Embedding

#For cleaning the tweets
import re
import string
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from nltk.tokenize import TweetTokenizer
from nltk.stem.lancaster import LancasterStemmer
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

#for metrics
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, f1_score

import nltk
nltk.download('punkt')

#warnings
import warnings
warnings.filterwarnings('ignore') 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Loading our datasets

In [ ]:
#Load the dataset
data = pd.read_csv('/content/Final-Remote-Learning-csv')

Previewing our data

In [ ]:
#Previewing the first ten records in our dataset
data.head(10)

,Unnamed: 0,Unnamed: 0.1,Tweets,user,location,id,lens,date,place,coordinates,lang,source,likes,retweets
0,0,0,b'ONLINE LEARNING |\nBen Troy Njue hosts Prof....,KBCTelevision,"Nairobi, Kenya",1427541371418783761,190,2021-08-17 08:02:40,NaN,NaN,en,TweetDeck,0,1
1,1,1,"b""Virtual school is back at Jinja city's educa...",felixadvise,"Jinja, Uganda",1427539081316216853,299,2021-08-17 07:53:34,NaN,NaN,en,Twitter Web App,6,3
2,2,2,"b""@KBCChannel1 @ManagementUNI the best in onl...",Daisywanga2,"Nairobi, Kenya",1427533647549247494,89,2021-08-17 07:31:58,NaN,NaN,en,Twitter for Android,2,0
3,3,3,b'Check out this course on TinyML (https://t.c...,VickTheGeek,"Nairobi, Kenya",1427516048178393106,192,2021-08-17 06:22:02,NaN,NaN,en,Twitter for Android,0,0
4,4,4,b'Want to learn more about genre as well as st...,WANTMofficial,Kenya,1427496748075954208,195,2021-08-17 05:05:21,NaN,NaN,en,Twitter Web App,1,0
5,5,5,b'Why #PedaLAfrica Online?\nIt has been design...,PASGR_,"Nairobi, Kenya",1427210497947021320,282,2021-08-16 10:07:53,NaN,NaN,en,Hootsuite Inc.,6,7
6,6,6,b'Group 6: Postgraduate learners embraced onli...,ciryan_nelson,Nairobi,1427202024886706176,239,2021-08-16 09:34:13,NaN,NaN,en,Twitter for iPhone,4,5
7,7,7,b'Micro- teaching as one of the online teachin...,stellah_kaviti,"Nairobi, Kenya",1427194709974396935,244,2021-08-16 09:05:09,NaN,NaN,en,Twitter for Android,5,7
8,8,8,b'PedaL online innovations offers better oppor...,PASGR_,"Nairobi, Kenya",1427176471144062988,260,2021-08-16 07:52:41,NaN,NaN,en,Hootsuite Inc.,9,11
9,9,9,b'Although learning from home has become more ...,alubusiness,Rwanda,1427171276267921410,272,2021-08-16 07:32:02,NaN,NaN,en,Sprout Social,8,1


In [ ]:
#Previewing the last ten records in our dataset
data.tail(10)

,Unnamed: 0,Unnamed: 0.1,Tweets,user,location,id,lens,date,place,coordinates,lang,source,likes,retweets
218,29,29,"b'""Education today is more expensive than it w...",FarajaDTv,"Kampala, Uganda",1425028145187540993,244,2021-08-10 09:36:00,NaN,NaN,en,Twitter for Android,14,9
219,30,30,b'Every Ugandan child will have coverage in th...,ota1978,"Kampala, Uganda",1425023561933148161,296,2021-08-10 09:17:47,NaN,NaN,en,Twitter for Android,3,3
220,31,31,b'Covid 19 has genuinely affected the educatio...,FarajaDTv,"Kampala, Uganda",1425022166056243201,302,2021-08-10 09:12:14,NaN,NaN,en,Twitter for Android,1,1
221,32,32,b'@EliKatunguka Indeed lack of e-learning infr...,allan_pincus,Kampala,1424977965012692999,255,2021-08-10 06:16:36,NaN,NaN,en,Twitter for iPhone,0,0
222,33,33,b'Whenever I open e-learning https://t.co/2R4C...,Theebenga,"Nairobi, Kenya",1424806252291534856,50,2021-08-09 18:54:16,NaN,NaN,en,Twitter for iPhone,0,0
223,34,34,"b""@nbstv But at times, Bill's are passed in ra...",mutrody,"Kampala, Uganda",1424801513734647814,271,2021-08-09 18:35:27,NaN,NaN,en,Twitter for Android,1,0
224,35,35,"b""@ntvuganda The teeth that can not move in un...",KansiimeGreens2,NaN,1424794370759290885,220,2021-08-09 18:07:04,Place(_api=<tweepy.api.API object at 0x7fb294a...,NaN,en,Twitter for Android,0,0
225,36,36,"b'Magut, H &amp; Mulinge, A. (2021). Impact of...",MauriceAmutabi,Kenya,1424722319705055232,232,2021-08-09 13:20:45,NaN,NaN,en,WordPress.com,1,1
226,37,37,"b""@isaacfuture001 \xf0\x9f\x94\x85Covid period...",Marvin_Myco,"Kampala, Uganda",1424721171774377988,245,2021-08-09 13:16:12,NaN,NaN,en,Twitter for Android,1,0
227,38,38,"b'The polices, that were developed using an or...",SEAD_Rwanda,"Kigali, Rwanda",1424641552945750016,285,2021-08-09 07:59:49,NaN,NaN,en,Twitter for iPhone,3,0


**Accessing basic information about our dataset**

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228 entries, 0 to 227
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    228 non-null    int64  
 1   Unnamed: 0.1  228 non-null    int64  
 2   Tweets        228 non-null    object 
 3   user          228 non-null    object 
 4   location      222 non-null    object 
 5   id            228 non-null    int64  
 6   lens          228 non-null    int64  
 7   date          228 non-null    object 
 8   place         15 non-null     object 
 9   coordinates   0 non-null      float64
 10  lang          228 non-null    object 
 11  source        228 non-null    object 
 12  likes         228 non-null    int64  
 13  retweets      228 non-null    int64  
dtypes: float64(1), int64(6), object(7)
memory usage: 25.1+ KB


In [ ]:
#Checking the shape of our dataset
print('This dataset has ' + str(data.shape[0]) + ' rows, and ' + str(data.shape[1]) + ' columns')

This dataset has 228 rows, and 14 columns


#**Data Cleaning**

**Validity**

In [ ]:
#We need to drop columns that will not be in use
data.drop(['Unnamed: 0', 'Unnamed: 0.1','place','lens', 'coordinates',
           'lang','retweets'], inplace= True, axis = 1)

**Consistency**

In [ ]:
#we will look for the number of rows with duplicate records
print('Number of rows with duplicated values',data.duplicated().sum())

#we then drop the rows with duplicated records
data.drop_duplicates(inplace=True)
print('After dropping there are',data.duplicated().sum(), "Duplicated rows")

print('Shape of dataframe after dropping duplicated rows', data.shape)

Number of rows with duplicated values 87
After dropping there are 0 Duplicated rows
Shape of dataframe after dropping duplicated rows (141, 7)


**Completeness**

In [ ]:
#We shall drop the null values
data.isnull().sum()

Tweets      0
user        0
location    3
id          0
date        0
source      0
likes       0
dtype: int64

In [ ]:
#We shall drop the null values 
data= data.dropna()

In [ ]:
#Confirming that our dataset has no null values
data.isnull().sum()

Tweets      0
user        0
location    0
id          0
date        0
source      0
likes       0
dtype: int64

**Uniformity**

In [ ]:
data.columns=data.columns.str.strip().str.lower().str.replace(' ','_').str.replace('(','').str.replace(')','')
data.columns

Index(['tweets', 'user', 'location', 'id', 'date', 'source', 'likes'], dtype='object')

# Text Preprocessing

In [ ]:
#Create a class with all function to clean our tweets
class CleanText(BaseEstimator, TransformerMixin):
    def remove_mentions(self, input_text):
        return re.sub(r'@\w+', '', input_text)
    
    def remove_urls(self, input_text):
        return re.sub(r'http.?://[^\s]+[\s]?', '', input_text)
    
    def emoji_oneword(self, input_text):
        # By compressing the underscore, the emoji is kept as one word
        return input_text.replace('_','')
    def remove_emoji(self, input_text):
      #data['clean_tweet']=data['clean_tweet'].apply(nfx.remove_emojis,nfx.remove_stopwords)
      return input_text()
    
    def remove_punctuation(self, input_text):
        # Make translation table
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')  # Every punctuation symbol will be replaced by a space
        return input_text.translate(trantab)
    def remove_digits(self, input_text):
        return re.sub('\d+', '', input_text)
    
    def to_lower(self, input_text):
        return input_text.lower()
    
    def remove_stopwords(self, input_text):
        stopwords_list = stopwords.words('english')
        # Some words which might indicate a certain sentiment are kept via a whitelist
        whitelist = ["n't", "not", "no"]
        words = input_text.split() 
        clean_words = [word for word in words if (word not in stopwords_list or word in whitelist) and len(word) > 1] 
        return " ".join(clean_words) 
    
    def stemming(self, input_text):
        porter = PorterStemmer()
        words = input_text.split() 
        stemmed_words = [porter.stem(word) for word in words]
        return " ".join(stemmed_words)
    
    def fit(self, X, y=None, **fit_params):
        return self
    
    def transform(self, X, **transform_params):
        clean_X = X.apply(self.remove_mentions).apply(self.remove_urls).apply(self.emoji_oneword).apply(self.remove_punctuation).apply(self.remove_digits).apply(self.to_lower).apply(self.remove_stopwords).apply(self.stemming)
        return clean_X

In [ ]:
data.head()

,tweets,user,location,id,date,source,likes
0,b'ONLINE LEARNING |\nBen Troy Njue hosts Prof....,KBCTelevision,"Nairobi, Kenya",1427541371418783761,2021-08-17 08:02:40,TweetDeck,0
1,"b""Virtual school is back at Jinja city's educa...",felixadvise,"Jinja, Uganda",1427539081316216853,2021-08-17 07:53:34,Twitter Web App,6
2,"b""@KBCChannel1 @ManagementUNI the best in onl...",Daisywanga2,"Nairobi, Kenya",1427533647549247494,2021-08-17 07:31:58,Twitter for Android,2
3,b'Check out this course on TinyML (https://t.c...,VickTheGeek,"Nairobi, Kenya",1427516048178393106,2021-08-17 06:22:02,Twitter for Android,0
4,b'Want to learn more about genre as well as st...,WANTMofficial,Kenya,1427496748075954208,2021-08-17 05:05:21,Twitter Web App,1


In [ ]:
#Removing emojis
# This code snippet encodes all the data into ASCII (American Standard Code for Information Interchange) values and ignore if the 
# data can not be encoded. After encoding it tries to decode them all again because all the emojis were ignored in the encoding 
# process. So now we have all the data without emojis.
data['tweets'] = data['tweets'].apply(lambda x: x.encode('ascii', 'ignore').decode('ascii'))

In [ ]:
#Intialize and call the class and taking a sample to confirm that the tweets have been cleaned
clean= CleanText()
sr_clean = clean.fit_transform(data.tweets)
sr_clean.sample(5)

112                 best learn onlin exam platform kenya
19                                      onlin learn baba
118    youthvib nna kenneth nchristoph niwagira richa...
136    kyambogo univers parent buy key learn gadget c...
0      onlin learn nben troy njue host prof washingto...
Name: tweets, dtype: object

In [ ]:
#Imputing null values
empty_clean = sr_clean == ''
print('{} records have no words left after text cleaning'.format(sr_clean[empty_clean].count()))
sr_clean.loc[empty_clean] = '[no_text]'

0 records have no words left after text cleaning


In [ ]:
#Convert to df
data['clean_tweet'] = sr_clean
data.head()

,tweets,user,location,id,date,source,likes,clean_tweet
0,b'ONLINE LEARNING |\nBen Troy Njue hosts Prof....,KBCTelevision,"Nairobi, Kenya",1427541371418783761,2021-08-17 08:02:40,TweetDeck,0,onlin learn nben troy njue host prof washingto...
1,"b""Virtual school is back at Jinja city's educa...",felixadvise,"Jinja, Uganda",1427539081316216853,2021-08-17 07:53:34,Twitter Web App,6,virtual school back jinja citi educ hub academ...
2,"b""@KBCChannel1 @ManagementUNI the best in onl...",Daisywanga2,"Nairobi, Kenya",1427533647549247494,2021-08-17 07:31:58,Twitter for Android,2,best onlin learn research carriculum
3,b'Check out this course on TinyML (https://t.c...,VickTheGeek,"Nairobi, Kenya",1427516048178393106,2021-08-17 06:22:02,Twitter for Android,0,check cours tinyml read infer microcontrol ten...
4,b'Want to learn more about genre as well as st...,WANTMofficial,Kenya,1427496748075954208,2021-08-17 05:05:21,Twitter Web App,1,want learn genr well studi win exampl check do...


In [ ]:
#Checking the most frequent word
a = data['clean_tweet'].str.cat(sep=' ')
words = nltk.tokenize.word_tokenize(a)
word_dist = nltk.FreqDist(words)
dff = pd.DataFrame(word_dist.most_common(), 
                    columns=['Word', 'Frequency'])
dff['Word_Count'] = dff.Word.apply(len)
dff[:5]

,Word,Frequency,Word_Count
0,learn,166,5
1,onlin,83,5
2,xf,80,2
3,digit,37,5
4,educ,28,4
